# 03d: Regression Algorithms

## วัตถุประสงค์การเรียนรู้
- เข้าใจหลักการของ Regression algorithms
- รู้จัก Linear, Polynomial, Ridge, Lasso และ Random Forest Regression
- เข้าใจ metrics สำหรับ Regression
- สามารถเลือกใช้อัลกอริทึมที่เหมาะสม

---

## 1. Regression คืออะไร?

- การทำนายค่าต่อเนื่อง (continuous values)
- ผลลัพธ์เป็นตัวเลข (ไม่ใช่หมวดหมู่)
- ตัวอย่าง: ราคาบ้าน, อุณหภูมิ, อัตราดอกเบี้ย

### การใช้งาน
- Price Prediction
- Sales Forecasting  
- Risk Assessment
- Time Series Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# สร้างข้อมูลตัวอย่าง
np.random.seed(42)
X, y = make_regression(n_samples=300, n_features=1, noise=20, random_state=42)

plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6)
plt.xlabel('Feature')
plt.ylabel('Target')
plt.title('Regression Sample Data')
plt.grid(True, alpha=0.3)
plt.show()

print(f"จำนวนข้อมูล: {X.shape[0]}")
print(f"Target range: {y.min():.2f} to {y.max():.2f}")

## 2. Linear Regression

### หลักการ
- หาเส้นตรงที่ fit กับข้อมูลดีที่สุด
- สมการ: y = mx + b
- ใช้ Least Squares method
- เร็ว, ง่าย, ตีความได้

### ข้อดี/ข้อเสีย
- **ข้อดี**: เร็ว, ตีความง่าย, ไม่ overfitting
- **ข้อเสีย**: ไม่จัดการ non-linear ได้

In [ ]:
# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# ประเมินผล
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print("Linear Regression Results:")
print(f"MSE: {mse_lr:.2f}")
print(f"R²: {r2_lr:.3f}")
print(f"MAE: {mae_lr:.2f}")
print(f"Slope: {lr_model.coef_[0]:.2f}")
print(f"Intercept: {lr_model.intercept_:.2f}")

# แสดงผล
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, alpha=0.6, label='Actual')
plt.scatter(X_test, y_pred_lr, alpha=0.6, color='red', label='Predicted')
plt.plot(X_test, y_pred_lr, 'r-', alpha=0.8)
plt.xlabel('Feature')
plt.ylabel('Target')
plt.title('Linear Regression Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Polynomial Regression

### หลักการ
- ขยาย Linear Regression ด้วย polynomial features
- สมการ: y = ax² + bx + c
- จัดการ non-linear relationships ได้
- ระวัง overfitting กับ degree สูง

In [ ]:
# สร้างข้อมูล non-linear
np.random.seed(42)
X_nonlinear = np.sort(np.random.uniform(-2, 2, 100)).reshape(-1, 1)
y_nonlinear = 0.5 * X_nonlinear.ravel() ** 2 + 0.3 * X_nonlinear.ravel() + np.random.normal(0, 0.3, 100)

X_train_nl, X_test_nl, y_train_nl, y_test_nl = train_test_split(
    X_nonlinear, y_nonlinear, test_size=0.2, random_state=42
)

# ทดสอบ polynomial degrees ต่างๆ
degrees = [1, 2, 3, 5]
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, degree in enumerate(degrees):
    # สร้าง polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X_train_poly = poly_features.fit_transform(X_train_nl)
    X_test_poly = poly_features.transform(X_test_nl)
    
    # ฝึก model
    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train_nl)
    
    # ทำนาย
    y_pred_poly = poly_model.predict(X_test_poly)
    r2_poly = r2_score(y_test_nl, y_pred_poly)
    
    # สร้างเส้นโค้งสำหรับแสดงผล
    X_plot = np.linspace(-2, 2, 100).reshape(-1, 1)
    X_plot_poly = poly_features.transform(X_plot)
    y_plot_poly = poly_model.predict(X_plot_poly)
    
    # แสดงผล
    axes[i].scatter(X_train_nl, y_train_nl, alpha=0.6, label='Train')
    axes[i].scatter(X_test_nl, y_test_nl, alpha=0.6, color='red', label='Test')
    axes[i].plot(X_plot, y_plot_poly, 'g-', linewidth=2, label='Prediction')
    axes[i].set_title(f'Degree {degree}, R² = {r2_poly:.3f}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Ridge Regression (L2 Regularization)

### หลักการ
- เพิ่ม penalty term เพื่อลด overfitting
- L2 penalty: λ * Σ(wi²)
- ทำให้ coefficients เล็กลง
- เหมาะสำหรับ multicollinearity

In [ ]:
# สร้างข้อมูลที่มี multicollinearity
np.random.seed(42)
n_samples = 100
X_multi = np.random.randn(n_samples, 5)
# สร้าง correlated features
X_multi[:, 1] = X_multi[:, 0] + 0.5 * np.random.randn(n_samples)
X_multi[:, 2] = X_multi[:, 0] - 0.3 * np.random.randn(n_samples)

# สร้าง target
true_coef = np.array([1.5, -2.0, 0.5, 0.8, -1.2])
y_multi = X_multi.dot(true_coef) + 0.1 * np.random.randn(n_samples)

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42
)

# เปรียบเทียบ Linear vs Ridge
alphas = [0.1, 1.0, 10.0, 100.0]
results = {'Linear': LinearRegression()}

for alpha in alphas:
    results[f'Ridge (α={alpha})'] = Ridge(alpha=alpha)

# ฝึกและประเมิน
comparison_results = pd.DataFrame()

for name, model in results.items():
    model.fit(X_train_multi, y_train_multi)
    y_pred = model.predict(X_test_multi)
    
    metrics = {
        'R²': r2_score(y_test_multi, y_pred),
        'MSE': mean_squared_error(y_test_multi, y_pred),
        'Coef_Sum': np.sum(np.abs(model.coef_))
    }
    
    comparison_results[name] = metrics

print("Linear vs Ridge Comparison:")
print(comparison_results.round(3))

## 5. Lasso Regression (L1 Regularization)

### หลักการ
- L1 penalty: λ * Σ|wi|
- ทำ feature selection อัตโนมัติ
- บาง coefficients จะเป็น 0
- เหมาะสำหรับ sparse data

In [ ]:
# เปรียบเทียบ Ridge vs Lasso
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=0.1)

ridge_model.fit(X_train_multi, y_train_multi)
lasso_model.fit(X_train_multi, y_train_multi)

# แสดง coefficients
coef_comparison = pd.DataFrame({
    'True': true_coef,
    'Ridge': ridge_model.coef_,
    'Lasso': lasso_model.coef_
}, index=[f'Feature {i+1}' for i in range(5)])

print("Coefficient Comparison:")
print(coef_comparison.round(3))

# แสดงผลในรูปกราฟ
coef_comparison.plot(kind='bar', figsize=(10, 6))
plt.title('Coefficient Comparison: Ridge vs Lasso')
plt.ylabel('Coefficient Value')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ประเมินผล
ridge_pred = ridge_model.predict(X_test_multi)
lasso_pred = lasso_model.predict(X_test_multi)

print(f"\nRidge R²: {r2_score(y_test_multi, ridge_pred):.3f}")
print(f"Lasso R²: {r2_score(y_test_multi, lasso_pred):.3f}")
print(f"\nNumber of zero coefficients in Lasso: {np.sum(np.abs(lasso_model.coef_) < 1e-5)}")

## 6. Random Forest Regression

### หลักการ
- Ensemble ของ Decision Trees สำหรับ regression
- จัดการ non-linear relationships ได้ดี
- ให้ feature importance
- ลด overfitting

In [ ]:
# Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_multi, y_train_multi)
rf_pred = rf_model.predict(X_test_multi)

print("Random Forest Results:")
print(f"R²: {r2_score(y_test_multi, rf_pred):.3f}")
print(f"MSE: {mean_squared_error(y_test_multi, rf_pred):.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': [f'Feature {i+1}' for i in range(5)],
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# แสดงผล
plt.figure(figsize=(8, 5))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance - Random Forest')
plt.show()

## 7. Regression Metrics

### หลัก Metrics
- **MAE**: Mean Absolute Error
- **MSE**: Mean Squared Error  
- **RMSE**: Root Mean Squared Error
- **R²**: Coefficient of Determination

### การเลือก Metric
- **MAE**: ทนต่อ outliers
- **MSE/RMSE**: penalty outliers มากกว่า
- **R²**: อธิบายความแปรปรวน

In [ ]:
# เปรียบเทียบ metrics ทั้งหมด
models_final = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

final_results = pd.DataFrame()

for name, model in models_final.items():
    model.fit(X_train_multi, y_train_multi)
    y_pred = model.predict(X_test_multi)
    
    metrics = {
        'MAE': mean_absolute_error(y_test_multi, y_pred),
        'MSE': mean_squared_error(y_test_multi, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_multi, y_pred)),
        'R²': r2_score(y_test_multi, y_pred)
    }
    
    final_results[name] = metrics

print("Model Comparison - All Metrics:")
print(final_results.round(3))

# แสดงผลในรูปกราฟ
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

metrics_to_plot = ['MAE', 'MSE', 'RMSE', 'R²']
for i, metric in enumerate(metrics_to_plot):
    ax = axes[i//2, i%2]
    final_results.loc[metric].plot(kind='bar', ax=ax)
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## สรุป

### Regression Algorithms
- **Linear**: เร็ว, ง่าย, linear relationships
- **Polynomial**: จัดการ non-linear ได้
- **Ridge**: ลด overfitting, multicollinearity
- **Lasso**: Feature selection, sparse data
- **Random Forest**: ความแม่นยำสูง, non-linear

### Metrics
- **MAE**: ทนต่อ outliers
- **MSE/RMSE**: penalty ขนาดใหญ่
- **R²**: อธิบายความแปรปรวน

### Next: Regression LAB